In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.optim as optim
from torchvision import models
from torchvision.models.vgg import VGG


import numpy as np 
import time

import numpy as np

In [ ]:
vgg16 = models.vgg16(pretrained=True)
vgg16_bn = models.vgg16_bn(pretrained=True)

In [21]:
class FCN32s(nn.Module):
    def __init__(self, model, num_classes):
        super(FCN32s, self).__init__()
        self.num_classes = num_classes
        self.model = model
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.classifier = nn.Conv2d(32, num_classes, kernel_size=1)

    def forward(self, x):
        output = self.model(x)
        x5 = output['x5']

        score = nn.BatchNorm2d(nn.ReLU(self.deconv1(x5), inplace=True))
        score = nn.BatchNorm2d(nn.ReLU(self.deconv2(score), inplace=True))
        score = nn.BatchNorm2d(nn.ReLU(self.deconv3(score), inplace=True))
        score = nn.BatchNorm2d(nn.ReLU(self.deconv4(score), inplace=True))
        score = nn.BatchNorm2d(nn.ReLU(self.deconv5(score), inplace=True))
        score = self.classifier(score)

        return score        

In [20]:
class FCN16s(nn.Module):
    def __init__(self, model, num_classes):
        super(FCN16s, self).__init__()
        self.num_classes = num_classes
        self.model = model
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.classifier = nn.Conv2d(32, num_classes, kernel_size=1)

    def forward(self, x):
        output = self.model(x)
        x5 = output['x5']
        x4 = output['x4']

        score = nn.ReLU(self.deconv1(x5), inplace=True)
        score = nn.BatchNorm2d(score + x4)
        score = nn.BatchNorm2d(nn.ReLU(self.deconv2(score), inplace=True))
        score = nn.BatchNorm2d(nn.ReLU(self.deconv3(score), inplace=True))
        score = nn.BatchNorm2d(nn.ReLU(self.deconv4(score), inplace=True))
        score = nn.BatchNorm2d(nn.ReLU(self.deconv5(score), inplace=True))
        score = self.classifier(score)

        return score        

In [19]:
class FCN8s(nn.Module):
    def __init__(self, model, num_classes):
        super(FCN8s, self).__init__()
        self.num_classes = num_classes
        self.model = model
        self.deconv1 = nn.ConvTranspose2d(512, 512, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv2 = nn.ConvTranspose2d(512, 256, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv3 = nn.ConvTranspose2d(256, 128, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv4 = nn.ConvTranspose2d(128, 64, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.deconv5 = nn.ConvTranspose2d(64, 32, kernel_size=3, stride=2, padding=1, dilation=1, output_padding=1)
        self.classifier = nn.Conv2d(32, num_classes, kernel_size=1)

    def forward(self, x):
        output = self.model(x)
        x5 = output['x5']
        x4 = output['x4']
        x3 = output['x3']

        score = nn.ReLU(self.deconv1(x5), inplace=True)
        score = nn.BatchNorm2d(score + x4)
        score = nn.ReLU(self.deconv2(x4), inplace=True)
        score = nn.BatchNorm2d(score + x3)
        score = nn.BatchNorm2d(nn.ReLU(self.deconv3(score), inplace=True))
        score = nn.BatchNorm2d(nn.ReLU(self.deconv4(score), inplace=True))
        score = nn.BatchNorm2d(nn.ReLU(self.deconv5(score), inplace=True))
        score = self.classifier(score)

        return score        

In [ ]:
class FCN(nn.Module):
    def __init__(self):
        super(FCN, self)).__init__()
        
    def forward(self, x):
        pass

In [ ]:
class VGGNet(nn.Module):
    def __init__(self, cfg, batch_norm):
        super(VGGNet, self).__init__()

    def _make_layers(self, cfg, batch_norm=False):
        layers = []
        for v in cfg:
            if v == 'M':
                layers += [nn.MaxPool2d(kernel_size=2, stride=2)]
            else:
                conv2d = nn.Conv2d(in_channels, v, kernel_size=3, padding=1)
                if batch_norm:
                    layers += [conv2d, nn.BatchNorm2d(v), nn.ReLU(inplace=True)]
                else:
                    layers += [conv2d, nn.ReLU(inplace=True)]
                in_channels = v

        return nn.Sequential(*layers)

In [17]:
vgg16.load_state_dict

<bound method Module.load_state_dict of VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel

## Reference
- [Fully Convolutional Networks for Semantic Segmentation](https://gaussian37.github.io/vision-segmentation-fcn/)